This is a python notebook made by **Yoonsoo P. Bach** to do photometry to the data obtained from the Seoul National University Astronomical Observatory (SNUO, also known as SAO, which I'd avoid due to the possible confusion with [Smithsonian Astrophysical Observatory](https://www.cfa.harvard.edu/sao)).

Observation was made on 2019-06-02 using the 1-m telescope at SNUO, using non-sidereal tracking mode for **(155140) 2005UD**.

It is the second of four notebooks:

1. ``Reducer``
    - preprocessing, CR rejection, and WCS implementation.
2. ``Photometer``
    - ephemerides/PS1 query, centroid stars, Pillbox Aperture photometry, zeropoint fitting.
        * It's results are in the ``phot_targ.csv``, and summarized in ``main.pdf``.
3. ``latex_generator`` 
    - generates the report LaTeX file and you may compile it.

In [1]:
%load_ext version_information
import time
now = time.strftime("%Y-%m-%d %H:%M:%S (%Z = GMT%z)")
print(f"This notebook was generated at {now} ")

vv = %version_information scipy, numpy, matplotlib, pandas, astropy, astroquery, photutils, version_information
for i, pkg in enumerate(vv.packages):
    print(f"{i} {pkg[0]:10s} {pkg[1]:s}")

This notebook was generated at 2019-09-18 18:52:55 (KST = GMT+0900) 
0 Python     3.7.3 64bit [GCC 7.3.0]
1 IPython    7.6.1
2 OS         Linux 4.15.0 58 generic x86_64 with debian buster sid
3 scipy      1.3.0
4 numpy      1.16.4
5 matplotlib 3.1.0
6 pandas     0.24.2
7 astropy    3.2.1
8 astroquery 0.3.10.dev5707
9 photutils  0.7
10 version_information 1.0.3


This is a data reducer for SNUO 1m observation. You can get the bleeding-edge versions from these links: [SNUO1Mpy](https://github.com/ysBach/SNUO1Mpy), [ysfitsutilpy](https://github.com/ysBach/ysfitsutilpy), and [ysphotutilpy](https://github.com/ysBach/ysphotutilpy).

But I recommend you to use the "frozen" version of those packages included in the attachment, because they may be updated with backward-incompatible changes.

In [2]:
import copy
import warnings
import logging
import logging.handlers
from pathlib import Path

import numpy as np
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.modeling.functional_models import GAUSSIAN_SIGMA_TO_FWHM
from astropy.stats import sigma_clipped_stats
from astropy.table import Table, hstack, vstack
from astropy.time import Time
from astropy.visualization import simple_norm
from matplotlib import pyplot as plt
from matplotlib import rcParams
from matplotlib.gridspec import GridSpec
from matplotlib.ticker import MultipleLocator, FormatStrFormatter
%matplotlib inline
from photutils.aperture import CircularAnnulus as CAn
from photutils.aperture import CircularAperture as CAp
from scipy.optimize import curve_fit
import gc

import snuo1mpy as snuo
import ysfitsutilpy as yfu
import ysphotutilpy as ypu

logger = logging.getLogger(__name__)
formatter = logging.Formatter('[%(asctime)s][%(levelname)s|%(filename)s:%(lineno)s] >> %(message)s')
streamHandler = logging.StreamHandler()
fileHandler = logging.FileHandler('./phot.log')
streamHandler.setFormatter(formatter)
fileHandler.setFormatter(formatter)
logger.addHandler(streamHandler)
logger.addHandler(fileHandler)
logger.setLevel(level=logging.DEBUG)
logger.propagate = False

warnings.filterwarnings('ignore', append=True, category=UserWarning)
warnings.filterwarnings('ignore', append=True, category=RuntimeWarning)

plt.style.use('default')
rcParams.update({'font.size': 12})

# Our object (will be queried to JPL HORIZONS)
OBJID = '155140'

# It is used as a rough estimate, so no need to be accurate:
PIX2ARCSEC = 0.31*u.arcsec

#####################################################################
# Settings for simplifying the code.

# Used for any `astropy.SkyCoord` object:
SKYC_KW = dict(unit=u.deg, frame='icrs')

# RectangularAperture does not have 'exact' yet.
# So to reduce warning messages, use subpixel:
PILL_KW = dict(method='subpixel', subpixels=32)

# Initial guess of FWHM in pixel
FWHM_INIT = 8

# The parameters used for centering target & stars.
# positions for ``sky_annulus can be dummy,
# cuz it will be updated by ``ypu.find_center_2dg`` iteratively.
SKY_ANNULUS = CAn([0, 0], r_in=4*FWHM_INIT, r_out=6*FWHM_INIT)
CENTER_KW = dict(cbox_size=8*FWHM_INIT, csigma=5., maxiters=10,
                 sky_annulus=SKY_ANNULUS,
                 max_shift_step=FWHM_INIT/2, max_shift=FWHM_INIT,
                 atol_shift=1.e-4)

# For drawing report figures:
CBAR_KW = dict(orientation='vertical', shrink=0.85)
ISOPH_KW = dict(colors='k', linewidths=(1,), linestyles='--')
REPORTDIR = Path("report")
FIGDIR = REPORTDIR / "figures"
yfu.mkdir(REPORTDIR)
yfu.mkdir(FIGDIR)
#####################################################################


# Functions to simplify the main code
def astro_image(fpath):
    """ Set parameters based on SNUO 1m telescope.
    You may have to manually tune this part for different facility.
    """
    from astropy.wcs import WCS
    ccd = yfu.load_ccd(fpath)
    hdr = ccd.header
    wcs = WCS(hdr)
    error = yfu.make_errmap(ccd=ccd,
                            gain_epadu=hdr["GAIN"],
                            rdnoise_electron=hdr["RDNOISE"],
                            flat_err=0)
    crota = yfu.wcs_crota(wcs, degree=True)

    return ccd, hdr, wcs, error, crota


def organize_ps1_query(ps1, ccd, error):
    ''' Organize the queried table & do centroiding to stars
    You may have to manually tune this part for different facility.
    '''
    # Organize ps1 query and check whether nearby PS1 object
    # near the target (query center):
    select_kws = dict(filter_names=["g", "r", "i"], n_mins=[3, 3, 1])
    isnear = ypu.organize_ps1_and_isnear(ps1,
                                         header=ccd.header,
                                         bezel=100,
                                         del_flags=[0, 1, 7, 8, 9],
                                         nearby_obj_minsep=5*FWHM_INIT*PIX2ARCSEC,
                                         group_crit_separation=5*FWHM_INIT,
                                         select_filter_kw=select_kws,
                                         drop_by_Kron=True,
                                         calc_JC=True)

    # Initialize the centroid-related columns:
    cols = ["xcentroid", "ycentroid", "shift_x", "shift_y", "shift",
            "theta_deg", "theta", "constant", "amplitude", "x_stddev", "y_stddev"]
    for c in cols:
        ps1.queried[c] = np.nan
        ps1.queried[c].format = "%.3f"

    # Calculate centroid from the image based on queried RA/DEC
    star_positions = []
    star_pos_history = []
    star_cuts = []
    star_ecuts = []
    star_fits = []
    star_models = []

    for i, row_star in enumerate(ps1.queried):
        pos = (row_star["x"], row_star["y"])
        newpos, shift, fd = ypu.find_center_2dg(ccd=ccd,
                                                position_xy=pos,
                                                error=error,
                                                **CENTER_KW,
                                                verbose=False,
                                                full=True)

        row_star["xcentroid"] = newpos[0]
        row_star["ycentroid"] = newpos[1]
        row_star["shift_x"] = newpos[0] - pos[0]
        row_star["shift_y"] = newpos[1] - pos[1]
        row_star["shift"] = shift
        mod = fd["fit_models"]
        for k, v in zip(mod.param_names, mod.parameters):
            if k in cols:
                row_star[k] = v
        row_star["theta_deg"] = np.rad2deg(mod.theta.value)

        star_cuts.append(fd["cuts"])
        star_fits.append(fd["fits"])
        star_ecuts.append(fd["e_cuts"])
        star_pos_history.append(fd["positions"])
        star_models.append(mod)
        star_positions.append(newpos)

    return (ps1.queried, star_positions, star_pos_history, 
            star_cuts, star_ecuts, star_fits, star_models, isnear)


def organize_ephemerides(obj, exptimes):
    ''' Add light-time corrected jd and trail length (pixel)
    '''
    eph = obj.query(depoch=50)
    jd = Time(eph["datetime_jd"], format='jd')
    lt = np.array(eph["lighttime"])*u.s
    eph["jd_target"] = jd - lt + (exptimes/2) * u.s 

    dRA = ((eph["RA_rate"]) * (exptimes) * u.s).to(u.arcsec)
    dDEC = ((eph["DEC_rate"]) * (exptimes) * u.s).to(u.arcsec)
    pos_1 = SkyCoord(ra=eph["RA"] - dRA/2,
                     dec=eph["DEC"] - dDEC/2,
                     **SKYC_KW)
    pos_2 = SkyCoord(ra=eph["RA"] + dRA/2,
                     dec=eph["DEC"] + dDEC/2,
                     **SKYC_KW)
    eph["trail"] = (pos_1.separation(pos_2)).to(u.arcsec) / PIX2ARCSEC
    return eph


def starfit_sigclip(stars, sigma=3, maxiters=5):
    sigclip_kw = dict(sigma=sigma, maxiters=maxiters, std_ddof=1)
    _, sig_x, _ = sigma_clipped_stats(stars["x_stddev"], **sigclip_kw)
    _, sig_y, _ = sigma_clipped_stats(stars["y_stddev"], **sigclip_kw)
    _, theta, _ = sigma_clipped_stats(stars["theta_deg"], **sigclip_kw)
    return sig_x, sig_y, theta


# A very dirty function...
def make_report_fig(ccd, ap, an, fwhm, ephem_theta, fit_theta,
                    fit_cut, fit_ecut, fit_image, 
                    fit_model, fit_positions, zp, dzp, objname,
                    cat_mag=None, cat_emag=None, 
                    cat_color=None, cat_ecolor=None):
    cutccd = copy.deepcopy(fit_cut)
    ecutccd = copy.deepcopy(fit_ecut)
    fitted = fit_image.copy()
    fitmod = fit_model.copy()
    xyvals = fit_positions.copy()
    ap = copy.deepcopy(ap)
    an = copy.deepcopy(an)

    ap_moved_1 = ypu.ap_to_cutout_position(ap, cutccd)
    cut_an = ypu.cutout_from_ap(an, ccd)
    ap_moved_2 = ypu.ap_to_cutout_position(ap, cut_an)
    an_moved_2 = ypu.ap_to_cutout_position(an, cut_an)

    norm = simple_norm(cutccd.data, 'sqrt')

    fig = plt.figure(figsize=(13, 6))
    gs = GridSpec(2, 4)
    ax_o = fig.add_subplot(gs[0, 0])    # 'o'riginal & centroid trace
    ax_f = fig.add_subplot(gs[0, 1])    # 'f'itted Gaussian
    ax_r = fig.add_subplot(gs[0, 2])    # 'r'esidual / photometric_error
    ax_a = fig.add_subplot(gs[1, 0])    # 'a'perture and 'a'nnulus
    ax_s = fig.add_subplot(gs[1, 1:3])  # 's'ky-histogram
    ax_e = fig.add_subplot(gs[:, 3])    # 'e'xplanatory

    half_isophote = (np.max(fitted) + fitmod.constant.value)/2
    im_o = ax_o.imshow(cutccd.data, norm=norm, origin='lower')

    for i, xy in enumerate(xyvals):
        xy_cut = cutccd.to_cutout_position(xy)
        if i == 0:
            label = f"Initial"
            mdict = dict(marker='x', color='r')
        elif i == len(xyvals)-1:
            label = f"{i}"
            mdict = dict(marker='+', color='r')
        else:
            mdict = dict(marker='.', color='r')
        ax_o.plot(*xy_cut, **mdict)

    ax_o.text(s=f"({xyvals[-1][0]:.2f}, {xyvals[-1][1]:.2f})",
              x=0, y=0, color='w')
    ax_o.contour(fitted, levels=[half_isophote, ], **ISOPH_KW)
    ax_o.set(title="Centering (x..+)")
    yfu.colorbaring(fig, im=im_o, ax=ax_o, **CBAR_KW)

    im_f = ax_f.imshow(fitted, norm=norm, origin="lower")
    ax_f.contour(fitted, levels=[half_isophote, ], **ISOPH_KW)
    ax_f.plot([], [], 'k--', lw=1, label="amp/2")
    ax_f.plot([], [], 'w-', lw=1, label="aperture")
    ax_f.legend(loc=8, frameon=False, fontsize=10)
    ax_f.set(title="2D Gaussian Fit")
    yfu.colorbaring(fig, im=im_f, ax=ax_f, **CBAR_KW)

    resid = (cutccd.data - fitted) / ecutccd.data
    im_r = ax_r.imshow(resid, vmin=-3, vmax=+3, origin="lower")
    ax_r.set(title=r"(data $-$ fit)/pix_err")
    yfu.colorbaring(fig=fig, ax=ax_r, im=im_r, extend='both',
                    label=r"$\sigma_\mathrm{pix} $", **CBAR_KW)

    for ax in [ax_o, ax_f, ax_r]:
        ap_moved_1.plot(axes=ax, color='w', lw=1)

    im_a = ax_a.imshow(cut_an.data, norm=norm, origin="lower")
    ap_moved_2.plot(axes=ax_a, color='w', lw=1)
    an_moved_2.plot(axes=ax_a, color='r', lw=1)
    yfu.colorbaring(fig=fig, ax=ax_a, im=im_a, **CBAR_KW)

    sky = ypu.sky_fit(ccd=ccd, annulus=an)
    m = sky["msky"][0]
    s = sky["ssky"][0]
    nsky = sky["nsky"][0]
    nrej = sky["nrej"][0]
    nall = nsky + nrej
    vals = np.arange(max(0, int(m - 5*s)), m + 5*s, 1)
    h = ax_s.hist(ypu.annul2values(ccd, an), bins=vals)
    ax_s.grid(which='major', ls='-')
    ax_s.axvline(m - 3*s, color='k', ls="--")
    ax_s.axvline(m, color='k', ls="-")
    ax_s.axvline(m + 3*s, color='k', ls="--")
    ax_s.errorbar(m, h[0].max(), xerr=s, marker='o', capsize=3, elinewidth=1)
    ax_s.set(xlabel="pixel value in annulus [ADU]",
             ylabel="# pix in annulus")

    fit_theta = (fit_theta + 180)%360 - 180
    ephem_theta = (ephem_theta + 180)%360 - 180
#     dtheta = ((fit_theta - np.rad2deg(fitmod.theta.value)) + 180)%360 - 180
    dtheta = ((np.rad2deg(fitmod.theta.value) - ephem_theta) + 180)%360 - 180

    ax_e.axis('off')
    info_wcs = ("* From catalog/ephemerides\n"
                + f"RA [deg] = {row_ephem['RA']:09.5f}\n"
                + f"DEC[deg] = {row_ephem['DEC']:+9.5f}\n"
                + f"pos_init = ({xyvals[0][0]:.2f}, {xyvals[0][1]:.2f})\n"
                + f"ephemerides theta ~ {ephem_theta:+.1f} deg\n"
                )
    if cat_mag is not None:
        info_wcs += (  f"mag   = {cat_mag:.3f} (err {cat_emag:.3f})\n"
                     + f"color = {cat_color:.3f} (err {cat_ecolor:.3f})\n")
    info_wcs += "\n"
    
    info_gauss = ("* 2D Gaussian Fit\n"
                  + f"pos_fit  = ({xyvals[-1][0]:.2f}, {xyvals[-1][1]:.2f})\n"
                  + f"shift    = ({xyvals[-1][0] - xyvals[0][0]:+.2f}, "
                  + f"{xyvals[-1][1] - xyvals[0][1]:+.2f})\n"
                  + f"bkg   = {fitmod.constant.value:.2f}\n"
                  + f"amp   = {fitmod.amplitude.value:.2f}\n"
                  + f"sigma = ({fitmod.x_stddev.value:.2f}, {fitmod.y_stddev.value:.2f})\n"
                  + f"theta = {fitmod.theta.value:+.4f} rad "
                  + f"= {np.rad2deg(fitmod.theta.value):+.2f} deg\n"
                  + f"  (from positive x-axis)\n"
                  + f"dtheta = {dtheta:+.2f} deg\n"
                  + "  (:= theta_Gauss_fit - theta_ephem)\n\n"
                 )

    info_synth = ("* Synthetic Aperture from All Objects:\n"
                  + f"FWHM   = {fwhm:.3f} pix ~ {fwhm*PIX2ARCSEC.value:.2f}'' \n"
                  + f"theta  = {fit_theta:+.2f} deg\n\n"
                 )
    info_sky = ("* Sky Annulus\n"
                + f"msky  = {m:.3f}\n"
                + f"ssky  = {s:.3f}\n"
                + f"n_all = {nall:.0f}\n"
                + f"n_rej = {nrej:.0f}\n"
                + f"n_sky = {nsky:.0f}\n"
                )
    
    inf_zp = ("* Zero Point\n"
             + f"zp = {zp:.3f} (err {dzp:.3f})")
    
    ax_e.text(s=info_wcs+info_gauss+info_synth+info_sky+inf_zp,
              x=0, y=0, fontsize=10, fontfamily="monospace")

    fig.subplots_adjust(top=0.90)
    plt.suptitle(objname)
    plt.tight_layout()
    return fig


def zpplot(ps1_star, phot_star):
    fig = plt.figure(figsize=(12, 5))
    gs = GridSpec(3, 3)
    # NOTE: Z = R - R_inst = zp + 1st ext
    ax_r = fig.add_subplot(gs[:2, 0])   # R_inst VS R plot
    ax_z = fig.add_subplot(gs[2, 0], sharex=ax_r)    # R VS Z plot
    ax_c = fig.add_subplot(gs[:, 1:])    # C(g-r) VS Z plot
    errbfmt = dict(marker='x', ls='', capsize=5, elinewidth=0.5)

    m = phot_star["mag"]
    dm = phot_star["merr"]
    R = ps1_star["Rmag"]
    dR = ps1_star["dRmag"]
    Cgr = ps1_star["C_gr"]
    dCgr = ps1_star["dC_gr"]
    zp = R - m
    dzp = np.sqrt(dm**2 + dR**2)
    szp = np.std(zp, ddof=1)
    ps1_star["zp"] = zp    # inplace addition of a column
    ps1_star["dzp"] = dzp  # inplace addition of a column
    zp_fit = np.average(zp, weights=(1/dzp**2))
    dzp_fit = np.sqrt(1 / np.sum(1/dzp**2))
    mzp = np.mean(zp)
    
    p_rr, _ = curve_fit(linf, R, m, sigma=dm, absolute_sigma=True)
    p_cz, _ = curve_fit(linf, Cgr, R - m, sigma=dzp, absolute_sigma=True)
    
    mm = np.linspace(R.min(), R.max(), 2)
    cc = np.linspace(Cgr.min(), Cgr.max(), 2)
    
    ax_r.errorbar(R, m, yerr=dm, xerr=dR, **errbfmt)
    ax_r.plot(mm, linf(mm, *p_rr), 'k-', label=f"y={p_rr[0]:.4f}x + {p_rr[1]:.3f}")
    ax_r.plot(mm, mm - zp_fit, 'r-', label=f"y=x + {zp_fit:.3f} (see right)")
    ax_r.set(ylabel="R_inst", title="Linearity Curve")
    ax_r.legend(loc=2, framealpha=0, fontsize=10)
    for j, (x_i, y_i) in enumerate(zip(R, m)):
        ax_r.text(x=x_i, y=y_i, s=j+1)
    
    ax_z.errorbar(R, zp, yerr=dR, **errbfmt)
    ax_z.axhline(zp_fit, color='r', ls='-', lw=1)
    ax_z.axhline(zp_fit + dzp_fit, color='r', ls='--', lw=1, label=None) 
    ax_z.axhline(zp_fit - dzp_fit, color='r', ls='--', lw=1, label=None)
    ax_z.axhline(mzp + szp, color='b', ls=':', lw=1, label=None) 
    ax_z.axhline(mzp - szp, color='b', ls=':', lw=1, label=None)
    ax_z.set(xlabel="R from PS1 g/r (Tonry+ 2012)",
             ylabel="R - R_inst")
    for j, (x_i, y_i) in enumerate(zip(R, zp)):
        ax_z.text(x=x_i, y=y_i, s=j+1)

    ax_c.errorbar(Cgr, zp, yerr=dzp, xerr=dCgr, **errbfmt)
    ax_c.axhline(zp_fit, color='r', ls='-', lw=1, label=f"wieghted avg = {zp_fit:.3f}")
    ax_c.axhline(mzp, color='b', ls='-', lw=1, label=f"simple avg = {mzp:.3f}")
    ax_c.axhline(zp_fit + dzp_fit, color='r', ls='--', lw=1, label=f"err = {dzp_fit:.3f}") 
    ax_c.axhline(zp_fit - dzp_fit, color='r', ls='--', lw=1, label=None)
    ax_c.axhline(mzp + szp, color='b', ls=':', lw=1, label=f"std = {szp:.3f}") 
    ax_c.axhline(mzp - szp, color='b', ls=':', lw=1, label=None)
    ax_c.plot(cc, linf(cc, *p_cz), 'k-',
              label=f"y={p_cz[0]:.3f}x + {p_cz[1]:.3f}")
    ax_c.legend(loc=2, framealpha=0, ncol=3)
    ax_c.set(xlabel="g - r from PS1",
             title="Z = R - R_inst = (k + k''X)C + (zero + k'X)")
    for j, (x_i, y_i) in enumerate(zip(Cgr, zp)):
        ax_c.text(x=x_i, y=y_i, s=j+1)
    
    yfu.linearticker([ax_r, ax_z, ax_c],
                    xmajlocs=[0.2, 0.2, 0.1],
                    xminlocs=[0.1, 0.1, 0.05],
                    ymajlocs=[0.2, 0.05, 0.05],
                    yminlocs=[0.1, 0.01, 0.01],
                    xmajfmts=["%.1f", "%.1f", "%.1f"],
                    ymajfmts=["%.1f", "%.2f", "%.2f"])
    
    plt.tight_layout()
    return fig, zp_fit, dzp_fit, mzp, szp, p_rr[0], p_cz[0]

    
def linf(x, a, b):
    return a*x + b

In [3]:
SUMMARYPATH = Path('.')/"summary_reduced_all.csv"
EPHEMPATH = Path('.')/"ephem.csv"

if not SUMMARYPATH.exists():
    _summarylist = list(Path('.').glob("**/summary_reduced.csv"))
    summary = vstack([Table.read(ff) for ff in _summarylist])
    summary = summary[2:]  # First two are dummy by accident...
    summary.write(SUMMARYPATH)

summary = Table.read(SUMMARYPATH)

epochs = Time(summary["DATE-OBS"], format='fits').jd

if not EPHEMPATH.exists():
    obj = ypu.HorizonsDiscreteEpochsQuery(targetname=OBJID,
                                          location=snuo.SITE_HORIZONS, 
                                          epochs=epochs,
                                          id_type="smallbody")
    eph = organize_ephemerides(obj, exptimes=summary["EXPTIME"])
    eph["file"] = np.array(summary["file"])
    eph.write(EPHEMPATH, format="ascii.csv", overwrite=True)

eph_all = Table.read(EPHEMPATH)
    
# The values for the target:
for c in ["xcentroid", "ycentroid", "fwhm", "fwhm_arcsec", "n_stars", "isnear"]:
    eph_all[c] = np.nan

In [4]:
def reduced_mag(mag, r_hel, r_obs):
    return mag - 5 * np.log10(r_hel * r_obs)

n_chop = 50

for i_eph in range(len(eph_all)//n_chop + 1):
    phottargpath = Path('.')/f"phot_targ_{i_eph+1}.csv"
    if phottargpath.exists():
        continue
    
    phot_targs = []
    eph = eph_all[n_chop*i_eph:n_chop*(i_eph+1)]
    for row_ephem in eph:
        fpath = Path(row_ephem["file"])
        if not fpath.exists():
            logger.info(f"NO FILE: {fpath}")
            continue
        ccd, hdr, wcs, error, crota = astro_image(fpath)
        now = time.strftime("%Y-%m-%d %H:%M:%S (%Z = GMT%z)")
        print(f"{now}")
        logger.info(fpath)

        phot_star_path = fpath.parent / f"{fpath.stem}_ps1.csv"
        pos_targ_init = SkyCoord(row_ephem["RA"], 
                                 row_ephem["DEC"],
                                 **SKYC_KW).to_pixel(wcs)
        rad_fov = yfu.fov_radius(header=hdr)  # `~Quantity` (u.deg)

#         print("01 - Fitting target...")
        logger.info("01 - Fitting target...")
        pos_targ_fit, shift, fd = ypu.find_center_2dg(ccd=ccd,
                                                      position_xy=pos_targ_init,
                                                      error=error,
                                                      **CENTER_KW,
                                                      verbose=False,
                                                      full=True)
        mod_targ = fd["fit_models"]
        pars_targ = fd["fit_params"]
        xys_targ = fd["positions"]
        fit_targ = fd["fits"]
        cut_targ = fd["cuts"]
        ecut_targ = fd["e_cuts"]

        logger.info("02 - Querying and Fitting stars...")

        # radius is arounded, hopefully it works without any problem...
        # see --- https://github.com/astropy/astroquery/issues/1551
        ps1 = ypu.PanSTARRS1(ra=row_ephem["RA"]*u.deg,
                             dec=row_ephem["DEC"]*u.deg,
                             radius=np.around(2*rad_fov.to_value(u.deg), 6),
                             columns=["**", "+objID"],
                             column_filters={"rmag": "10.0..15.2"})
        results_ps1 = organize_ps1_query(ps1,
                                         ccd=ccd,
                                         error=error)

        ps1_stars = results_ps1[0]
        star_positions = results_ps1[1]
        star_pos_history = results_ps1[2]
        star_cuts = results_ps1[3]
        star_ecuts = results_ps1[4]
        star_fits = results_ps1[5]
        star_models = results_ps1[6]
        isnear = results_ps1[7]
        
        pos_star_fit = np.array([ps1_stars["xcentroid"], ps1_stars["ycentroid"]]).T

        _, sig_y_fit, theta_fit = starfit_sigclip(ps1_stars)
        # larger sigma, smaller sigma, theta in degrees (CCW from image X axis)

        # smaller sigma as indicator of true FWHM
        fwhm = sig_y_fit*GAUSSIAN_SIGMA_TO_FWHM


        # Add columns to ephemerides
        row_ephem["xcentroid"] = pos_targ_fit[0]
        row_ephem["ycentroid"] = pos_targ_fit[1]
        row_ephem["fwhm"] = fwhm
        row_ephem["fwhm_arcsec"] = (fwhm*PIX2ARCSEC).value
        row_ephem["n_stars"] = len(ps1_stars)
        row_ephem["isnear"] = isnear

#         print("03 - Photometry (STAR --> ", end='')
        logger.info("03 - Photometry (STAR... ")
        # Set aperture/annulus for stars
        ap_star, an_star = ypu.pill_ap_an(pos_star_fit, fwhm, 
                                          trail=row_ephem["trail"],
                                          f_ap=(1.75, 1.75),
                                          theta=np.deg2rad(theta_fit))

        # Photometry to stars
        phot_star = ypu.apphot_annulus(ccd=ccd, 
                                       aperture=ap_star, 
                                       annulus=an_star, 
                                       error=error,
                                       **PILL_KW)
        phot_star = hstack([Table(ps1_stars), Table(phot_star)],
                           metadata_conflicts='silent')

#         print("ZP --> ", end='')
        logger.info("ZP... ")
        # Simple linear fit to get zp for color = 0 as reference and the slope (k_R + k"_R*X).
        fig, zp_fit, dzp_fit, mzp, szp, linSlope, zpSlope = zpplot(ps1_stars, phot_star)
        phot_star.write(phot_star_path, format="ascii.csv", overwrite=True)

        figpath = FIGDIR/Path("{}_{}.pdf".format(fpath.stem, "zeropoint"))
        plt.savefig(figpath)
    #     plt.savefig(FIGDIR/Path("{}_{}.png".format(fpath.stem, "zeropoint")))
        fig.clf()
        plt.close(fig)

        logger.info("TARGET)")

        ap_targ, an_targ = ypu.circ_ap_an(pos_targ_fit, fwhm=fwhm, f_ap=2)
        phot_targ = ypu.apphot_annulus(ccd=ccd, 
                                       aperture=ap_targ, 
                                       annulus=an_targ,
                                       error=error,
                                       **PILL_KW)
        phot_targ["linSlope"] = linSlope  # linearity curve slope (just for check)
        phot_targ["zp"] = zp_fit  # weighted mean
        phot_targ["dzp"] = dzp_fit
        phot_targ["mzp"] = mzp  # simple mean
        phot_targ["zpStd"] = szp
        phot_targ["zpSlope"] = zpSlope  # color VS zp slope
        phot_targ["m_std"] = phot_targ["mag"] + zp_fit
        phot_targs.append(phot_targ)    

        logger.info("04 - Drawing Report Figures")
        theta_eph = row_ephem["velocityPA"] + theta_fit + crota
        fig = make_report_fig(ccd, ap_targ, an_targ, fwhm,
                              fit_theta=theta_fit,
                              ephem_theta=theta_eph,
                              fit_cut=fd["cuts"], 
                              fit_ecut=fd["e_cuts"],
                              fit_image=fd["fits"], 
                              fit_model=fd["fit_models"], 
                              fit_positions=fd["positions"],
                              zp=zp_fit,
                              dzp=dzp_fit,
                              objname="(155140) 2005 UD"
                             )
        figpath = FIGDIR/Path("{}_{}.pdf".format(fpath.stem, "target"))
        plt.savefig(figpath)
    #     plt.savefig(FIGDIR/Path("{}_{}.png".format(fpath.stem, "target")))
        fig.clf()
        plt.close(fig)

        for j, row_star in enumerate(ps1_stars):
            objid = row_star["objID"]
            ap_j = ap_star[j]
            an_j = an_star[j]
            pos_j = star_pos_history[j]
            cut_j = star_cuts[j]
            ecut_j = star_ecuts[j] 
            fit_j = star_fits[j]
            mod_j = star_models[j]
            fig = make_report_fig(ccd, ap_j, an_j, fwhm,
                                  fit_theta=theta_fit,
                                  ephem_theta=theta_eph,
                                  fit_cut=cut_j, 
                                  fit_ecut=ecut_j,
                                  fit_image=fit_j, 
                                  fit_model=mod_j, 
                                  fit_positions=pos_j,
                                  objname=f"objID = {objid} ('{j+1}' in zero plot)",
                                  cat_mag=row_star["Rmag"],
                                  cat_emag=row_star["dRmag"],
                                  cat_color=row_star["C_gr"],
                                  cat_ecolor=row_star["dC_gr"],
                                  zp=row_star["zp"],
                                  dzp=row_star["dzp"]
                                 )
            figpath = FIGDIR/Path("{}_star_{}.pdf".format(fpath.stem, objid))
            plt.savefig(figpath)
    #         plt.savefig(FIGDIR/Path("{}_star_{}.png".format(fpath.stem, objid)))
            fig.clf()
            plt.close(fig)

        plt.close(plt.gcf())
        plt.close('all')
        gc.collect()
        logger.info("DONE\n")

    gc.collect()
    
    phot_targs_tab = hstack([eph, Table(vstack(phot_targs))], 
                            metadata_conflicts='silent')
    phot_targs_tab["m_red"] = reduced_mag(phot_targs_tab["m_std"], eph["r"], eph["delta"])
    phot_targs_tab["dm_red"] = np.sqrt(phot_targs_tab["merr"]**2 
                                       + phot_targs_tab["dzp"]**2)
    phot_targs_tab.write(phottargpath, overwrite=True)
    

2019-09-18 18:53:09 (KST = GMT+0900)


[2019-09-18 18:53:09,974][INFO|<ipython-input-4-9244a4090e02>:21] >> reduced/SNUO_STX16803-2005UD-1-1-20181013-184638-R-60.0.fits
[2019-09-18 18:53:10,067][INFO|<ipython-input-4-9244a4090e02>:30] >> 01 - Fitting target...
[2019-09-18 18:53:11,820][INFO|<ipython-input-4-9244a4090e02>:44] >> 02 - Querying and Fitting stars...


 33 objects remaining:  66 masked out of  99 based on 100-pixel bezel.
 27 objects remaining:   6 masked out of  33 based on DAOGROUP with 40.000-pixel critical separation..
 26 objects remaining:   1 masked out of  27 based on f_objID ([0, 1, 7, 8, 9]).
 19 objects remaining:   7 masked out of  26 based on the Kron magnitude criterion.
 16 objects remaining:   3 masked out of  19 based on o_['g', 'r', 'i']mag >= [3 3 1].


[2019-09-18 18:53:28,380][INFO|<ipython-input-4-9244a4090e02>:84] >> 03 - Photometry (STAR... 
[2019-09-18 18:53:28,914][INFO|<ipython-input-4-9244a4090e02>:101] >> ZP... 
[2019-09-18 18:53:29,777][INFO|<ipython-input-4-9244a4090e02>:112] >> TARGET)
[2019-09-18 18:53:29,972][INFO|<ipython-input-4-9244a4090e02>:129] >> 04 - Drawing Report Figures
[2019-09-18 18:53:59,596][INFO|<ipython-input-4-9244a4090e02>:183] >> DONE



2019-09-18 18:53:59 (KST = GMT+0900)


...

[2019-09-18 19:00:05,819][INFO|<ipython-input-4-9244a4090e02>:84] >> 03 - Photometry (STAR... 
[2019-09-18 19:00:06,324][INFO|<ipython-input-4-9244a4090e02>:101] >> ZP... 
[2019-09-18 19:00:07,080][INFO|<ipython-input-4-9244a4090e02>:112] >> TARGET)
[2019-09-18 19:00:07,269][INFO|<ipython-input-4-9244a4090e02>:129] >> 04 - Drawing Report Figures
[2019-09-18 19:00:35,427][INFO|<ipython-input-4-9244a4090e02>:183] >> DONE



2019-09-18 19:00:36 (KST = GMT+0900)


[2019-09-18 19:00:36,368][INFO|<ipython-input-4-9244a4090e02>:21] >> reduced/SNUO_STX16803-2005UD-1-1-20181013-185745-R-60.0.fits
[2019-09-18 19:00:36,463][INFO|<ipython-input-4-9244a4090e02>:30] >> 01 - Fitting target...
[2019-09-18 19:00:37,773][INFO|<ipython-input-4-9244a4090e02>:44] >> 02 - Querying and Fitting stars...


 32 objects remaining:  65 masked out of  97 based on 100-pixel bezel.
 26 objects remaining:   6 masked out of  32 based on DAOGROUP with 40.000-pixel critical separation..
 25 objects remaining:   1 masked out of  26 based on f_objID ([0, 1, 7, 8, 9]).
 18 objects remaining:   7 masked out of  25 based on the Kron magnitude criterion.
 15 objects remaining:   3 masked out of  18 based on o_['g', 'r', 'i']mag >= [3 3 1].


[2019-09-18 19:00:55,699][INFO|<ipython-input-4-9244a4090e02>:84] >> 03 - Photometry (STAR... 
[2019-09-18 19:00:56,162][INFO|<ipython-input-4-9244a4090e02>:101] >> ZP... 
[2019-09-18 19:00:56,902][INFO|<ipython-input-4-9244a4090e02>:112] >> TARGET)
[2019-09-18 19:00:57,143][INFO|<ipython-input-4-9244a4090e02>:129] >> 04 - Drawing Report Figures
[2019-09-18 19:01:26,055][INFO|<ipython-input-4-9244a4090e02>:183] >> DONE



In [36]:
PHOTSPATH = Path(".")/"phot_targ.csv"
if not PHOTSPATH.exists():
    import pandas as pd
    
    _phots = list(Path(".").glob("phot_targ_*.csv"))
    phots = pd.concat([pd.read_csv(fpath) for fpath in _phots])
    phots.to_csv(PHOTSPATH)

phots = Table.read(PHOTSPATH)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 6), sharex=True, sharey=True, gridspec_kw=None)

yfu.zimshow(axs[0], ccd.data)
axs[0].plot(ps1_stars["xcentroid"], ps1_stars["ycentroid"], 'ro', ms=5, mfc='none')
ap_star.plot(axes=axs[0])
ap_star.plot(axes=axs[1])
an_star.plot(axes=axs[0])

m = np.zeros_like(ccd.data)
for apm in ap_star.to_mask():
    m += apm.to_image(ccd.data.shape)
axs[1].imshow(m, origin="lower")
axs[1].plot(ap_star.positions[:,0], ap_star.positions[:,1], 'ro', ms=5, mfc='none')
# axs[0].set(xlim=(1100,1150), ylim=(1960,2020))
plt.tight_layout()
plt.show()

For debugging purpose, I used this to check the centroiding process:
```python
from astropy.visualization import simple_norm
cutccd = targ_cuts[-1]
norm = simple_norm(cutccd.data, 'sqrt')

fig, axs = plt.subplots(1, 2, figsize=(10, 4), sharex=False, sharey=False, gridspec_kw=None)

im = axs[0].imshow(cutccd.data, norm=norm, origin='lower')
xs_cut, ys_cut = cutccd.to_cutout_position((xs, ys))
axs[0].plot(xs_cut, ys_cut, 'r+')
for i, (xs_i, ys_i) in enumerate(zip(xs_cut, ys_cut)):
    if i == 0:
        label = f"{i+1} iter"
    else:
        label = f"{i+1}"
    axs[0].text(x=xs_i+3, y=ys_i, s=label)
yfu.colorbaring(fig, im=im, ax=axs[0], orientation='vertical')

im = axs[1].imshow(aimg["ccd"].data, norm=norm, origin="lower")
axs[1].plot(xs, ys, 'r+')
for i, (xs_i, ys_i) in enumerate(zip(xs, ys)):
    if i == 0:
        label = f"{i+1} iter"
    else:
        label = f"{i+1}"
    axs[1].text(x=xs_i+3, y=ys_i, s=label)
yfu.colorbaring(fig, im=im, ax=axs[1], orientation='vertical')


plt.tight_layout()
plt.show()
```

In [ ]:
now = time.strftime("%Y-%m-%d %H:%M:%S (%Z = GMT%z)")
print(f"This notebook was generated at {now} ")